<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Specify device
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# Import all necessary libraries.
import sys
import cv2

In [3]:
# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')

sys.path.append(NOMEROFF_NET_DIR)

In [4]:
# Import license plate recognition tools.
from NomeroffNet.YoloV5Detector import Detector
detector = Detector()
detector.load()

In [5]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft, getCvZoneRGB, convertCvZonesRGBtoBGR, reshapePoints
npPointsCraft = NpPointsCraft()
npPointsCraft.load()

Loading weights from checkpoint (/mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [6]:
from NomeroffNet.OptionsDetector import OptionsDetector

optionsDetector = OptionsDetector()
optionsDetector.load("modelhub://numberplate_options_uacustom")

NPOptionsNet(
  (inp_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_reg): Dropout(p=0.2, inplace=False)
  (fc1_reg): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_reg): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_reg): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_reg): Linear(in_features=256, out_features=9, bias=True)
  (dropout_line): Dropout(p=0.2, inplace=False)
  (fc1_line): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_line): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_line): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_line): Linear(i

In [7]:
from NomeroffNet.TextDetector import TextDetector
from NomeroffNet.TextPostprocessing import textPostprocessing

textDetector = TextDetector.get_static_module("eu")
textDetector.load("latest")

NPOcrNet(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [10]:
# Detect numberplate
img_path = 'images/example2.jpeg'
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

targetBoxes = detector.detect_bbox(img)
all_points = npPointsCraft.detect(img, targetBoxes,[5,2,0])

# cut zones
zonesRGB = [getCvZoneRGB(img, reshapePoints(rect, 1)) for rect in all_points]
zones = convertCvZonesRGBtoBGR(zonesRGB)

# predict zones attributes 
region_ids, count_lines, classification_score = optionsDetector.predict(zones, return_acc=True)
regions_confidences, count_lines_confidences = classification_score
print("[INFO] count lines local:", count_lines, count_lines_confidences)
print("[INFO] region names local:", region_names, regions_confidences)

count_lines, count_lines_confidences = optionsDetector.custom_count_lines_id_to_all_count_lines_with_confidences(
    count_lines, 
    count_lines_confidences)
region_ids, regions_confidences = optionsDetector.custom_regions_id_to_all_regions_with_confidences(
    region_ids, 
    regions_confidences)
region_names = optionsDetector.get_regions_label_global(region_ids)
print("[INFO] region ids global:", region_ids, count_lines)
print("[INFO] count lines global:", count_lines, count_lines_confidences)
print("[INFO] region names global:", region_names, regions_confidences)

# find text with postprocessing by standart
text_arr = textDetector.predict(zones)
text_arr = textPostprocessing(text_arr, region_names)
print("[INFO] predicted numberplate text:", text_arr)

2021-11-23 18:26:29,703 [172847] WARNING  py.warnings:109: [JupyterRequire] /mnt/data/var/www/nomeroff-net/NomeroffNet/nnmodels/numberplate_options_model.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x1 = functional.softmax(self.fc3_reg(x1))

2021-11-23 18:26:29,723 [172847] WARNING  py.warnings:109: [JupyterRequire] /mnt/data/var/www/nomeroff-net/NomeroffNet/nnmodels/numberplate_options_model.py:122: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x2 = functional.softmax(self.fc3_line(x2))



[INFO] count lines local: [1, 1] [[     0.8938    0.050205    0.055994]
 [    0.88538    0.057644    0.056977]]
[INFO] region names local: ['eu_ua_1995', 'eu_ua_1995'] [[    0.14451     0.10788    0.099108     0.15249      0.1018    0.092822    0.081487     0.11446     0.10543]
 [     0.1202     0.10456     0.10629     0.16024    0.098767    0.096377    0.083102     0.12225     0.10822]]
[INFO] region ids global: [4, 4] [1, 1]
[INFO] count lines global: [1, 1] [[0, 0.89380115, 0.050204515, 0.055994365], [0, 0.88537943, 0.0576439, 0.056976702]]
[INFO] region names global: ['eu', 'eu'] [[0, 0.14451261, 0.10788474, 0.09910769, 0.1524941, 0.10180455, 0, 0, 0.09282209, 0.08148744, 0.11446123, 0, 0.10542543, 0, 0, 0, 0], [0, 0.12020136, 0.104557455, 0.10629245, 0.16023916, 0.09876687, 0, 0, 0.09637705, 0.08310248, 0.12224638, 0, 0.10821677, 0, 0, 0, 0]]
[INFO] predicted numberplate text: ['RP70012', 'JJF509']
